# Porto Seguro's Safe Driving Prediction (AutoML Local Compute)

Porto Seguro, one of Brazil’s largest auto and homeowner insurance companies, completely agrees. Inaccuracies in car insurance company’s claim predictions raise the cost of insurance for good drivers and reduce the price for bad ones.

In the [Porto Seguro Safe Driver Prediction competition](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction), the challenge is to build a model that predicts the probability that a driver will initiate an auto insurance claim in the next year. While Porto Seguro has used machine learning for the past 20 years, they’re looking to Kaggle’s machine learning community to explore new, more powerful methods. A more accurate prediction will allow them to further tailor their prices, and hopefully make auto insurance coverage more accessible to more drivers.

Lucky for you, a machine learning model was built to solve the Porto Seguro problem by the data scientist on your team. The solution notebook has steps to load data, split the data into test and train sets, train, evaluate and save a LightGBM model that will be used for the future challenges.

#### Hint: use shift + enter to run the code cells below. Once the cell turns from [*] to [#], you can be sure the cell has run. 

## Import Needed Packages

Import the packages needed for this solution notebook. The most widely used package for machine learning is [scikit-learn](https://scikit-learn.org/stable/), [pandas](https://pandas.pydata.org/docs/getting_started/index.html#getting-started), and [numpy](https://numpy.org/). These packages have various features, as well as a lot of clustering, regression and classification algorithms that make it a good choice for data mining and data analysis.

In [18]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn import metrics

##  Get Azure ML Workspace to use

In [19]:
from azureml.core import Workspace, Dataset

# Get Workspace defined in by default config.json file
ws = Workspace.from_config()

## Load data from file into regular Pandas DataFrame

In [20]:

DATA_DIR = "../../../data/"
data_df = pd.read_csv(os.path.join(DATA_DIR, 'porto_seguro_safe_driver_prediction_train.csv'))

print(data_df.shape)
# print(data_df.describe())
data_df.head(5)

(595212, 59)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


## Split Data into Train and Validatation Sets

Partitioning data into training, validation, and holdout sets allows you to develop highly accurate models that are relevant to data that you collect in the future, not just the data the model was trained on. 

In machine learning, features are the measurable property of the object you’re trying to analyze. Typically, features are the columns of the data that you are training your model with minus the label. In machine learning, a label (categorical) or target (regression) is the output you get from your model after training it.

In [21]:
# Split in train/validation datasets (Validation=20%, Train=80%)
# Only split in test/train
train_df, validation_df = train_test_split(data_df, test_size=0.2, random_state=0)
train_df.describe()

# If seggregating the Y from X features
# features = data_df.drop('target', axis = 1)
#labels = np.array(data_df['target'])
#features_train, features_valid, labels_train, labels_valid = train_test_split(features, labels, test_size=0.2, random_state=0)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,476169.00,476169.00,476169.00,476169.00,476169.00,476169.00,476169.00,476169.00,476169.00,476169.00,...,476169.00,476169.00,476169.00,476169.00,476169.00,476169.00,476169.00,476169.00,476169.00,476169.00
mean,743752.39,0.04,1.90,1.36,4.42,0.42,0.41,0.39,0.26,0.16,...,5.44,1.44,2.87,7.54,0.12,0.63,0.55,0.29,0.35,0.15
std,429531.33,0.19,1.98,0.67,2.70,0.49,1.35,0.49,0.44,0.37,...,2.33,1.20,1.70,2.75,0.33,0.48,0.50,0.45,0.48,0.36
min,7.00,0.00,0.00,-1.00,0.00,-1.00,-1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,371729.00,0.00,0.00,1.00,2.00,0.00,0.00,0.00,0.00,0.00,...,4.00,1.00,2.00,6.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,743246.00,0.00,1.00,1.00,4.00,0.00,0.00,0.00,0.00,0.00,...,5.00,1.00,3.00,7.00,0.00,1.00,1.00,0.00,0.00,0.00
75%,1115723.00,0.00,3.00,2.00,6.00,1.00,0.00,1.00,1.00,0.00,...,7.00,2.00,4.00,9.00,0.00,1.00,1.00,1.00,1.00,0.00
max,1488027.00,1.00,7.00,4.00,11.00,1.00,6.00,1.00,1.00,1.00,...,19.00,10.00,13.00,23.00,1.00,1.00,1.00,1.00,1.00,1.00


## Train with Azure AutoML automatically searching for the 'best model' (Best algorithms and best hyper-parameters)

### List and select primary metric to drive the AutoML classification problem

In [22]:
from azureml.train import automl

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

# I'll use 'accuracy' as primary metric (Closer to 1.00 is better)

['AUC_weighted',
 'norm_macro_recall',
 'average_precision_score_weighted',
 'accuracy',
 'precision_score_weighted']

## Define AutoML Experiment settings

In [23]:
import logging

# You can provide additional settings as a **kwargs parameter for the AutoMLConfig object
# automl_settings = {
#      "whitelist_models": ['LightGBM']
# }

from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='classification',
                             primary_metric='AUC_weighted',                           
                             training_data=train_df,
                             validation_data=validation_df,
                             label_column_name="target",                                                    
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log='automated_ml_errors.log',
                             verbosity= logging.INFO,
                             enable_onnx_compatible_models=False
                             # **automl_settings
                             )

# Explanation of Settings: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings

# AutoMLConfig info on: 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig

## Run Experiment with multiple child runs under the covers

In [ ]:
from azureml.core import Experiment

experiment_name = "SDK_local_porto_seguro_driver_pred"
print(experiment_name)

experiment = Experiment(workspace=ws, 
                        name=experiment_name)

run = experiment.submit(automl_config, show_output=True)

SDK_local_porto_seguro_driver_pred
Running on local machine
Parent Run ID: AutoML_2e83951d-8c79-45fe-9761-683178449713

Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  Classes in the training data are imbalanced.
PARAMETERS:   Size of the smallest class : 17383, number of samples in the training data : 476169
              
TYPE:         Missing values imputation
STATUS:       PASSED
DESCRIPTION:  There were no missing values found in the training data.

TYPE:         High cardinali

## Explore results with Widget

In [ ]:
# Explore the results of automatic training with a Jupyter widget: https://docs.microsoft.com/en-us/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py
from azureml.widgets import RunDetails
RunDetails(run).show()

## Retrieve the 'Best' Scikit-Learn Model

In [ ]:
best_run, fitted_model = run.get_output()
print(best_run)
print('--------')
print(fitted_model)

## See files associated with the 'Best run'

In [ ]:
print(best_run.get_file_names())

# best_run.download_file('azureml-logs/70_driver_log.txt')

## Make Predictions

### Load Test Dataset from AML Workspace
ISSUE: The Test dataset doesn't have any label column...

In [ ]:
aml_test_dataset = ws.datasets['porto_seguro_safe_driver_prediction_test']

test_df = aml_test_dataset.to_pandas_dataframe()
# .to_pandas_dataframe().dropna()
print(test_df.shape)
test_df.head(5)

### Prep Validation or Test Data: Extract X values (feature columns) from test dataset and convert to NumPi array for predicting 

In [ ]:
import pandas as pd

#Remove Label/y column
if 'target' in validation_df.columns:
    y_validation_df = validation_df.pop('target')

x_validation_df = validation_df

x_validation_df.describe()

In [ ]:
y_validation_df.describe()

### Load model in memory

In [ ]:
# Load the model into memory
import joblib
fitted_model = joblib.load('model.pkl')
print(fitted_model)

### Make predictions in bulk

In [ ]:
# Try the best model making predictions with the test dataset
y_predictions = fitted_model.predict(x_validation_df)

print('10 predictions: ')
print(y_predictions[:10])

## Evaluate Model

Evaluating performance is an essential task in machine learning. In this case, because this is a classification problem, the data scientist elected to use an AUC - ROC Curve. When we need to check or visualize the performance of the multi - class classification problem, we use AUC (Area Under The Curve) ROC (Receiver Operating Characteristics) curve. It is one of the most important evaluation metrics for checking any classification model’s performance.

<img src="https://www.researchgate.net/profile/Oxana_Trifonova/publication/276079439/figure/fig2/AS:614187332034565@1523445079168/An-example-of-ROC-curves-with-good-AUC-09-and-satisfactory-AUC-065-parameters.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 12px; width: 320px; height: 239px;" />

### Calculate the Accuracy with Validation or Test Dataset

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy with Scikit-Learn model:')
print(accuracy_score(y_validation_df, y_predictions))


In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_validation_df, y_predictions)
print('AUC (Area Under the Curve) with Scikit-Learn model:')
metrics.auc(fpr, tpr)

# AUC with plain LightGBM was: 0.6374553321494826 